In [ ]:
import pandas as pd
import nltk
import string
import re
from nltk.stem.porter import PorterStemmer

nltk.download('stopwords')
from nltk.corpus import stopwords

nltk.download('punkt')
ps = PorterStemmer()

In [21]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score

In [4]:
file = pd.read_csv("/content/sample_data/Tweets.csv", encoding='latin-1')
file.shape

(14640, 15)

In [5]:
file.head(1)

,tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,570306133677760513,neutral,1.0,NaN,NaN,Virgin America,NaN,cairdin,NaN,0,@VirginAmerica What @dhepburn said.,NaN,2015-02-24 11:35:52 -0800,NaN,Eastern Time (US & Canada)


In [6]:
file = file[["airline_sentiment", "text"]]
file.sample()

,airline_sentiment,text
4124,negative,@united so I wasted 40mins filling in 2 online...


In [10]:
def clean_text(text):
    text = text.lower()
    text = re.sub(r'http.?://[^\s]+[\s]?', '', text)
    text = nltk.word_tokenize(text)
    y = []

    for i in text:
      if i not in stopwords.words('english'):
                  y.append(i)

    text = y[:]
    y.clear()

    for i in text:
            y.append(ps.stem(i))

    return" ".join(y)

In [11]:
file["text_cleaned"] = file["text"].apply(clean_text)

In [29]:
file.sample()

text         \
                  count unique   
airline_sentiment                
negative           9178   9087   
neutral            3099   3067   
positive           2363   2298   

                                                                           \
                                                                 top freq   
airline_sentiment                                                           
negative           @AmericanAir that's 16+ extra hours of travel ...    2   
neutral                                           @SouthwestAir sent    5   
positive                                            @JetBlue thanks!    5   

                  text_cleaned         \
                         count unique   
airline_sentiment                       
negative                  9178   9083   
neutral                   3099   3027   
positive                  2363   2262   

                                                                           
                                                                 top freq  
airline_sentiment                                                          
negative           @ americanair fyi ... call still get drop . ho...    2  
neutral            @ jetblu 's ceo battl appeas passeng wall stre...    8  
positive                                            @ jetblu thank !   12

In [27]:
tf_vec = TfidfVectorizer(max_features=3000)
X = tf_vec.fit_transform(file["text_cleaned"]).toarray()
Y = file["airline_sentiment"].values
# print(Y)
X.shape

(14640, 3000)

In [19]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=2)

In [24]:
model = MultinomialNB()
model.fit(X_train, Y_train)
Y_pred = model.predict(X_test)
print(accuracy_score(Y_test, Y_pred))

0.7219945355191257


In [26]:
from sklearn.ensemble import RandomForestClassifier

modelRF = RandomForestClassifier()
modelRF.fit(X_train,Y_train)
Y_predRF = modelRF.predict(X_test)
print(accuracy_score(Y_test,Y_predRF))

0.7469262295081968
